<a href="https://colab.research.google.com/github/ehudb9/N1X10_dev/blob/develop/document_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Documet Classification class:** 

__Author : Ehud Barda 

 gets path to file (mode=PDF)  or txt file (mode=TXT)and output set of index that describe and verify the file.
 procedure check that the file belongs to the patient, return the document relevant date (event date) and return document type out of a list of types (stored in file_type_list)

# installing

In [12]:
# Tesseract dependencies
!pip install pytesseract
!sudo apt install tesseract-ocr
!sudo apt install libtesseract-dev
# Poppler  dependencies
!apt-get install poppler-utils
!pip install pdf2image
!python -m pip install pypdf2
!pip install pdfminer
!pip install tika

from datetime import date
import pytesseract
import shutil
import os
import random
from google.colab import drive
try:
 from PIL import Image
except ImportError:
 import Image

from pdf2image import convert_from_path, convert_from_bytes
from IPython.display import display, Image

!pip install PyPDF2
from PyPDF2 import PdfFileReader
from google.colab import drive

from tika import parser
import PyPDF2
import re
import yaml
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import StringIO
import time
import csv
from io import StringIO
import pandas as pd
from os import listdir
from os.path import isfile, join, splitext

print('\n\nTesseract Version:', pytesseract.get_tesseract_version())

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libtesseract-dev is already the newest version (4.00~git2288-10f4998a-2).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
poppler-utils is already the newest version (0.62.0-2ubuntu2.12).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


Tesseract Version: 4.0.0-beta.1


# constans

CONSTANS: need to move to dict files and impot it to here

In [ ]:
# CONSTANS: need to move to dict files and impot it to here


procedure_types_EXP = {
    'Pathology and Surgery': ["Pathology", "CYTOPATHOLOGY", "Surgical Procedure", "SURGICAL", "Surgical", "PATHOLOGY"],
    "Medication prescriptions": ["Medication", "Medications"],
    "Imaging": ["CT", "XR", "NM", "FDG", "PET/CT", "PET", 'XR', 'MRI', 'PET CT', 'CT', 'DOTATATE'],
    "Blood Test": ["Collection Information", "METABOLIC", "LAB RESULT"],
    "Dr summaries": [""],
    "Hospital release forms": [""],
    "Medication prescriptions": [""],
}
TITLES = ["report", "summery", "result"]
DOC_DATE = ["date of report"]
date_of_procedure = ["Collection", "collected", "s" ]
institution_list = [""]

# document class





the class get path to pdf and open it to extract the indexes:

need to be change to .txt input withput haldle the pdf obj.

In [112]:
# the class get path to pdf and open it to extract the indexes:

# need to be change to .txt input withput haldle the pdf obj.

class DocumentClassification:
    """

    gets path to file (mode="PDF")  or txt file (mode="TXT")and output set of index that describe and verify the file.
    procedure check that the file belongs to the patient,
    return the document relevant date (event date) and
    return document type out of a list of types (stored in file_type_list)

    :param str: path to file
    :param (optional)str: patient full name
    """

    def __init__(self, path, file=None, patient_name=None, mode="TXT"):
        self.mode = mode
        self.path = path
        self.patient_name = patient_name
        page_num = 0
        
        if mode == "PDF":
          self.pdf_mode = "pdfObj"
          # Need to be replace with txt input
          with open(self.path, "rb") as self.f:
              reader = PyPDF2.PdfFileReader(self.f)
              page = reader.getPage(0)
              page_num = reader.getNumPages()
              self.read_pdf = page.extractText()

          if len(self.read_pdf) < 10:
              self.read_pdf = self.convert_pdf_to_txt(self.path)
              self.pdf_mode = "hackPdf"
              if len(self.read_pdf) < 10 or self.read_pdf[0] == "(":
                  # ocr
                  print("---------____needed to acrivate OCR mode____---------\n\n\n\n")
                  self.pdf_mode = "OCR"
                  # TBC on colab
                  self.read_pdf = self.ocr_convert_pdf_to_str(file)
        # if input TXT file
        else: 
          self.read_pdf = file
    
        #remove empty lines for better accuracy
        self.remove_empty_lines()

        self.doc_index = { 
            "patient_name": self.get_patient_name(), 
            "date_of_document": self.get_doc_date(),  #TBC
            "date_of_procedure": self.get_procedure_date(), #TBC
            "doctor_name": self.get_doctor_name(),
            # "department": self.get_department(),
            "institution": self.get_get_institution_name(),  #TBC
            "procedure_type": self.extract_type(),  #TBC
            "num_of_pages": page_num,  #need to add assignment when is TXT given
            "document_reference": self.path
        }

    def close_file(self):
        """

        close the file obj
        """
        self.f.close()

    def get_doc_index(self):
        """
        # to print use:
        # import yaml
        # print(yaml.dump(a.get_doc_index()))


        :return: dict: the given doc_index
        """
        return self.doc_index
        

    def remove_empty_lines(self):
        """
        rempves empty line in the text
        """
        lines = self.read_pdf.split("\n")
        non_empty_lines = [line for line in lines if line.strip() != ""]
        string_without_empty_lines = ""
        for line in non_empty_lines:
            string_without_empty_lines += line + "\n"

        self.read_pdf = string_without_empty_lines

    def _print_index(self):
        """

        prints the doc_index(JSON/DICT)
        """
        print(yaml.dump(self.get_doc_index()))

    def get_text(self) -> str:
        """
        :return: str: doc text - in lower case
        """
        return self.read_pdf.lower()

    def get_doctor_name(self):
        """
        :return: str doctor name
        """
        lines = "\n".join(self.read_pdf.splitlines()[2:20])
        pattern = re.compile(r'([A-Z]\w+[^:\n]+)\s(MD|M\.D\.)')
        search = re.search(pattern, lines)
        if search:
            return search.group()

    def get_get_institution_name(self):
        """
        # problem with read from pictures -TBC
        :return: str :  institution_name
        """
        
        pass

    def get_doc_date(self):
        pass

    def get_procedure_date(self):
        pass

    def convert_pdf_to_txt(self, path) -> str:
        """
        read pdf file and conevt the text to str using pdfminer
        :param path: file input
        :return: str: file's text in str
        """
        rsrcmgr = PDFResourceManager()
        retstr = StringIO()
        codec = 'utf-8'
        laparams = LAParams()
        device = TextConverter(rsrcmgr, retstr, laparams=laparams)
        fp = open(path, 'rb')
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        password = ""
        maxpages = 0
        caching = True
        pagenos = set()

        for page in PDFPage.get_pages(fp, pagenos, maxpages=maxpages, password=password, caching=caching,
                                      check_extractable=True):
            interpreter.process_page(page)

        text = retstr.getvalue()

        fp.close()
        device.close()
        retstr.close()
        return text

    def ocr_pdf_to_str(self, path):
        """
        MUST CONNECT TO OCR MODULE
        :return: str : file text
        """
        str_lst = ["CONNECT", "OCR", "!"]
        # img_lst = convert_pdf_to_image_list(path)
        # str_lst = convert_list_of_images_to_list_of_str(img_lst)
        txt = '\n'.join([i for i in str_lst[1:]])
        return txt

    def extract_type(self):
        """
        Go through the first 15 lines and get the type
        from a constant list of types

        :return: str : document type
        """
        lines = self.get_text().splitlines()
        num_line = 2
        types = {}
        for line in lines[2:15]:
            for type in procedure_types_EXP:
                for term in procedure_types_EXP[type]:
                    words = line.split()
                    for word in words:
                        if word == term.lower():
                            types[num_line] = type

            num_line += 1
        if len(types) == 0:
            return None
        return (min(types.items(), key=lambda x: x[0])[1])

    def search_line_index(self, word):
        """
        search for a given word in self.text
        return the line index where the word found. and -1 if wasn't found
        """
        pass

    def get_patient_name(self):
        """
        Check if a given patient name is correct
        and set the name if was't given a name
        """
        if self.patient_name is None:
            match = re.search(r'(patient|name|patient name): (\w+\s\w+)', self.read_pdf.lower())
            if match:
                return match.group(2)
            else:
                return "No name"
        name = self.patient_name.split()
        i = 0
        lines = self.get_text().splitlines()
        for line in lines[2:15]:
            list_line = line.split()
            for a in name:
                if a in list_line:
                    i +=1
            if i == len(name):
                return self.patient_name
            i = 0
        return "no name"


    #   for colab- calling ocr methods
    def ocr_convert_pdf_to_str(self, path):
        img_lst = self.convert_pdf_to_image_list(path)
        str_lst = self.convert_list_of_images_to_list_of_str(img_lst)
        txt = '\n'.join([i for i in str_lst[1:]])
        return txt


OCR FUNCTIONS:

In [ ]:
def convert_pdf_to_image_list(pdf_path):
    """
        this function converts pdf to a python list of <class 'PIL.PpmImagePlugin.PpmImageFile'> images.

        Parameters
        ----------
        pdf_path : string
            pdf_path is the local path of the pdf file on given machine.

        Returns
        -------
        list
            Python list of <class 'PIL.PpmImagePlugin.PpmImageFile'> images,
            where each image corresponds to a single pdf page.

        """
    pages = convert_from_path(pdf_path, 300)  # pages is a list of images
    return pages


def convert_list_of_images_to_list_of_str(img_lst):
    """
        this function converts a list of images to a list of strings.

        Parameters
        ----------
        img_lst : Python list of <class 'PIL.PpmImagePlugin.PpmImageFile'> images.

        Returns
        -------
        Python list
            Python list of strings corresponding to img_lst.

        """
    lst_of_str = []
    for img in img_lst:
        lst_of_str.append(pytesseract.image_to_string(img))
    return lst_of_str


def convert_list_of_images_to_list_of_xml(img_lst):
    """
        this function converts a list of images to a list of xml (type=byte).

        Parameters
        ----------
        img_lst : Python list of <class 'PIL.PpmImagePlugin.PpmImageFile'> images.

        Returns
        -------
        Python list
            Python list of xml(type=byte) corresponding to img_lst.

        """
    lst_of_xml = []
    for img in img_lst:
        lst_of_xml.append(pytesseract.image_to_pdf_or_hocr(img, extension='hocr'))  # Get HOCR output)
    return lst_of_xml


def convert_list_of_images_to_list_of_data(img_lst):
    """
        this function converts a list of images to a list of tables of data.

        Parameters
        ----------
        img_lst : Python list of <class 'PIL.PpmImagePlugin.PpmImageFile'> images.

        Returns
        -------
        Python list
            Python list of tables of data corresponding to img_lst.

        """
    lst_of_data = []
    for img in img_lst:
        lst_of_data.append(pytesseract.image_to_data(img))
    return lst_of_data

# files creator
running on a given path and creating CSV file with the result on document class


Callingand creating files:

In [273]:
class file_creator:
  """
  creat csv file with document indexes
  using Dpcument classification class 
  """
  def __init__(self, path=None):
    drive.mount('/content/drive')
    # Drive path:
    self.test_path = r'/content/drive/Shareddrives/co-op_projects/Sample documents/'
    self.main_dir = path
    if path == None:
      self.main_dir = self.test_path
    fields = ['patient_name',
              'date_of_document',
              'date_of_procedure',
              'doctor_name',
              'institution',
              'procedure_type',
              'num_of_pages',
              'document_reference',
              'Last modified',
              'Document']
    # rows = [[i for i in range(10)]]
    # name of csv file
    self.global_doc_index = self.main_dir +"/global_doc_index.csv"
    
    with open(self.global_doc_index, 'w') as self.csvfile:
      # creating a csv writer object
      csvwriter = csv.writer(self.csvfile)
        
      # writing the fields
      csvwriter.writerow(fields)
        
      # writing the data rows
      # csvwriter.writerows(rows)  
    

    self.sub_dirs = [join(self.main_dir , f) for f in listdir(self.main_dir) if not isfile(join(self.main_dir , f))]
    
    
  def list_sub_dir_fullpath(self):
    return [join(self.main_dir, f) for f in os.listdir(self.main_dir) if not isfile(join(self.main_dir, f))]

  def get_list_of_only_files(self):
    onlyfiles = [f for f in listdir(self.main_dir) if isfile(join(mypath, f))]
    return onlyfiles
        
  def doc_index_to_CSV(self, file):
    pass
  
  def run(self):
    # each type
    for dir in self.sub_dirs:
      # doc dirs
      doc_sub_dir = [join(dir , f) for f in listdir(dir) if not isfile(join(dir,f)) and not f.endswith(".ipynb_checkpoints") and not f.endswith(".pdf")]
      doc_names = [f for f in listdir(dir) if not isfile(join(dir,f)) and not f.endswith(".ipynb_checkpoints") and not f.endswith(".pdf")]
      for doc in doc_sub_dir:
        # print(doc)
        # print(doc_names[doc_sub_dir.index(doc)])
        
        #current ocr_txt
        onlyTxtFiles = [f for f in listdir(doc) if f.endswith(".txt")]
         
        if "OCR_txt_all.txt" in onlyTxtFiles:
          ocr_file = doc+"/OCR_txt_all.txt"
          with open(ocr_file, 'r') as file:
              ocr_txt ="".join(file.readlines())
          
          doc_class = DocumentClassification(doc+".pdf", file=ocr_txt, patient_name=None, mode='TXT')
          indexes = doc_class.doc_index
          now = datetime.now()
          # dd/mm/YY H:M:S
          dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
          indexes["Last modified"] = dt_string
          # TBC: write in the current doc path "doc_index.csv"
          indexes["Document"] = doc_names[doc_sub_dir.index(doc)]

          # field names
          fields = [key for key in indexes.keys()]
            
          # data rows of csv file
          rows = [[val for val in indexes.values()]]
            
          # name of csv file
          filename = doc+"/doc_index.csv"
          print("creating"+indexes["Document"]+ "  File")
          # writing to csv file
          with open(filename, 'w') as csvfile:
            # creating a csv writer object
            csvwriter = csv.writer(csvfile)
              
            # writing the fields
            csvwriter.writerow(fields)
              
            # writing the data rows
            csvwriter.writerows(rows)
          #  ADD row with the "indexes" to the global doc index
          global_csv_row = [val for val in indexes.values()]
          with open(self.global_doc_index, 'a') as csvfile:
            # creating a csv writer object
            csvwriter = csv.writer(csvfile)
            # writing the current index
            csvwriter.writerow(global_csv_row)


        # onlyTxtFiles = [f for f in listdir(p) if isfile(join(p, f)) and  f.endswith(".txt")]
        # # working_on_first_txt_file
        # working_on_first_txt_file = p+"/"+onlyTxtFiles[0]
        # print(working_on_first_txt_file)
        # with open(working_on_first_txt_file, 'r') as file:

  




In [274]:
a=file_creator()
print(a.run())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
creatingCT 2020-1-20  File
creatingCT19  File
creatingCT  2019-7-16  File
creatingCT 2020-3-26  File
creatingMRI 2019-1-29  File
creatingMRI 2017-12-18  File
creatingCT chest, abdomen, pelvis 07052020  File
creating01032020 CT-AbdomenPelvis wLiver Triphasic  File
creating01092020 Radiology  File
creating01102018 CT CHABDPEL WCON  File
creating04162019 NM-NM SIRSPH NM THRPY-ADM BREMSLIV SPCT  File
creating06132018 MR-Liver wwo Con  File
creating10212019 PET-NECK.CAP - FDG  File
creating10242017 MR RECTUM WWO CONTRAST  File
creating12272019 US-Extremity Lower Veins Bilateral  File
creating20160408_imaging_SM_1  File
creatingPET CT 5.27.2016  File
creatingAC PET_CT 2.19.2020  File
creatingAC CXR 11.2.2020  File
creatingCXR 12.11.2020  File
creatingAC PET_CT 9_19_2019  File
creatingAC CXR 6.24.2019  File
creatingCXR 2.11.2021  File
creatingCT chest abd 3.25.2019 

# TESTER:: RUN THE PROGRAM:: MAIN SECTION!!!

In [ ]:
a=file_creator()
a.run()

# draft --only for develop!

draft cells:

In [21]:
# a = DocumentClassification(r"C:\Users\ehudb\PycharmProjects\nixio test\Sample documents\Pathology\2016.4.14. Brigham slide review.pdf")
# # print(a.print_indexs)
# print(yaml.dump(a.get_doc_index()))
# a.close_file()

a = DocumentClassification(r"/content/drive/Shareddrives/co-op_projects/Sample documents/Pathology/2019.6.26. endobronchial bx.pdf" , mode="PDF")
#print(a._print_index())
print(yaml.dump(a.get_doc_index()))
# print(a.read_pdf)

a.close_file()

{date_of_document: null, date_of_procedure: null, doctor_name: 'Chalker, Richard Bruce,
    MD', document_reference: /content/drive/Shareddrives/co-op_projects/Sample documents/Pathology/2019.6.26.
    endobronchial bx.pdf, institution: null, num_of_pages: 3, patient_name: No name,
  procedure_type: Pathology and Surgery}



In [232]:
# working cell:
test_path = r'/content/drive/Shareddrives/co-op_projects/Sample documents/'

sub_dir = [join(test_path , f) for f in listdir(test_path) if not isfile(join(test_path , f))]

doc = r"/content/drive/Shareddrives/co-op_projects/Sample documents/Imaging/EM imaging"
# print(listdir(p))

# ocr_txt
ocr_file = p+"/OCR_txt_all.txt"
with open(ocr_file, 'r') as file:
    ocr_txt =" ".join(file.readlines())
r = DocumentClassification(p+".pdf", file=ocr_txt, patient_name=None, mode='TXT')
indexes = r.doc_index
now = datetime.now()
# dd/mm/YY H:M:S
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
indexes["Last modified"] = dt_string
indexes["Document"] = doc
# print(indexes.keys)
print(indexes)


#write in the current doc path "doc_index.csv"

# field names
fields = [key for key in indexes.keys()]
  
# data rows of csv file
rows = [[val for val in indexes.values()]]
  
# name of csv file
filename = doc+"/doc_index.csv"
  
# writing to csv file
with open(filename, 'w') as csvfile:
  # creating a csv writer object
  csvwriter = csv.writer(csvfile)
    
  # writing the fields
  csvwriter.writerow(fields)
    
  # writing the data rows
  csvwriter.writerows(rows)
  
  # TBC: ADD row with the "indexes" to the global doc index


# a = DocumentClassification(p+".pdf" , mode="PDF")
# #print(a._print_index())
# print(yaml.dump(a.get_doc_index()))

{'patient_name': 'No name', 'date_of_document': None, 'date_of_procedure': None, 'doctor_name': 'JOEL SHEINFELD, MD', 'institution': None, 'procedure_type': 'Imaging', 'num_of_pages': 0, 'document_reference': '/content/drive/Shareddrives/co-op_projects/Sample documents/Imaging/EM imaging.pdf', 'Last modified': '08/06/2021 17:45:21', 'Document': '/content/drive/Shareddrives/co-op_projects/Sample documents/Imaging/EM imaging'}


In [122]:
from datetime import datetime

# datetime object containing current date and time
now = datetime.now()
 
print("now =", now)

# dd/mm/YY H:M:S
dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
print("date and time =", dt_string)

now = 2021-06-08 15:17:28.055767
date and time = 08/06/2021 15:17:28


In [185]:
with open('indexes.csv', 'w') as f:
    for key in indexes.keys():
        f.write("%s, %s\n" % (key, indexes[key]))

In [227]:
# Python program to demonstrate
# writing to CSV


import csv
	
# field names
fields = [key for key in indexes.keys()]
	
# data rows of csv file
rows = [[test for i in range(10)]]
	
# name of csv file
filename = "doc_index.csv"
	
# writing to csv file
with open(filename, 'w') as csvfile:
	# creating a csv writer object
	csvwriter = csv.writer(csvfile)
		
	# writing the fields
	csvwriter.writerow(fields)
		
	# writing the data rows
	csvwriter.writerows(rows)


In [256]:
fields = [key for key in indexes.keys()]
len(fields)

10

In [189]:
fields = [val for val in indexes.values()]
fields
  

['No name',
 None,
 None,
 'JOEL SHEINFELD, MD',
 None,
 'Imaging',
 0,
 '/content/drive/Shareddrives/co-op_projects/Sample documents/Imaging/EM imaging.pdf',
 '08/06/2021 16:55:01']

In [ ]:
indexes


In [ ]:
print(ocr_txt)